In [26]:
# Библиотеки

from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.optim as optim
from torchdiffeq import odeint             # пока не понятно для чего

In [27]:
# Переменные

IS_RESTART = False
P_CUTOFF = 0.0                                                     # Непонятно для чего
N_EPOCH = 1000                                                     # Количество эпох. Максимальное
N_PLOT = 100                                                       # Частота формирования графиков. Через сколько эпох

# Есть большая вероятность, что нам не нужен будет OPT, и что строчку ниже нужно будет удалить
OPT = optim.AdamW([torch.tensor(1.0)], lr=0.001, betas=(0.9, 0.999), eps=1e-8)          # Оптимизатор
DATASIZE = 10                                                      # Размер датасетов?
TSTEP = 1                                                          # Шаг времени для датасетов? или типа их количество?
N_EXP_TRAIN = 6                                                    # Размер данных для обучения
N_EXP_TEST = 2                                                     # Размер даных для теста
N_EXP = N_EXP_TRAIN + N_EXP_TEST                                   # Общий размер данных
NOISE = 5e-2                                                       # ШУМ
NS = 5                                                             # Количество веществ
NR = 4                                                             # Количество хим. реакций
K = torch.tensor([0.1, 0.2, 0.13, 0.3], dtype=torch.float32)                                   # константы хим. реакций
ATOL = 0.00001                                                     # Параметр точности для ОДУ
RTOL = 0.01                                                        # Параметр точности для ОДУ

MAXITERS = 10000                                                   # Не понял для чего нужно

LB = 0.00001
UB = 1


In [28]:
u0_list = np.random.rand(N_EXP, NS).astype(np.float32)
u0_list[:, 0:2] += 0.2
u0_list[:, 2:] = 0.0
tspan = [0.0, DATASIZE * TSTEP]
tsteps = np.linspace(tspan[0], tspan[1], DATASIZE)               # возможно надо оставить просто DATASIZE
ode_data_list = np.zeros((N_EXP, NS, DATASIZE), dtype=np.float32)

In [29]:
print(tsteps)

[ 0.          1.11111111  2.22222222  3.33333333  4.44444444  5.55555556
  6.66666667  7.77777778  8.88888889 10.        ]


### Дальше пойдут функции

In [30]:
def true_ode_func(t, y, k):
    """
    Вычисляет производные системы дифференциальных уравнений.
    
    Параметры:
    t : float
        Время (не используется в данном уравнении, но необходим для совместимости с solve_ivp).
    y : array-like
        Вектор переменных (y[0], y[1], ..., y[4]).
    k : array-like
        Коэффициенты (k[0], k[1], ..., k[4]).

    Возвращает:
    dydt : numpy.ndarray
        Вектор производных.
    """
    dydt = np.zeros_like(y)
    dydt[0] = -2 * k[0] * y[0]**2 - k[1] * y[0]
    dydt[1] = k[0] * y[0]**2 - k[3] * y[1] * y[3]
    dydt[2] = k[1] * y[0] - k[2] * y[2]
    dydt[3] = k[2] * y[2] - k[3] * y[1] * y[3]
    dydt[4] = k[3] * y[1] * y[3]
    return dydt


In [ ]:
std_list = []
for i in range(N_EXP):
    u0 = u0_list[i, :]
    # Определяем функцию, описывающую ОДУ, и задачу
    prob_trueode = lambda t, y: true_ode_func(t, y=u0, k=K)  # trueODEfunc аналогична
    
    # Решение ОДУ с заданным алгоритмом и временем
    sol = solve_ivp(prob_trueode, tspan, u0, method='RK45', t_eval=tsteps)
    ode_data = sol.y  # Преобразуем в матрицу (в Julia это Array)
    
    # Добавляем шумы
    ode_data += np.random.randn(*ode_data.shape) * ode_data * NOISE
    
    # Сохраняем данные
    ode_data_list[i, :, :] = ode_data
    
    # Вычисляем max_min для текущих данных
    std_list.append(np.max(ode_data, axis=1) - np.min(ode_data, axis=1) + LB)

# Вычисляем y_std
std_matrix = np.column_stack(std_list)  # Аналог hcat в Julia
y_std = np.max(std_matrix, axis=1)

In [34]:
print(u0_list)
print()
print(u0_list[1, :])

[[1.0015455  1.0607156  0.         0.         0.        ]
 [0.98516953 0.6033745  0.         0.         0.        ]
 [0.42716482 0.6048892  0.         0.         0.        ]
 [0.9676658  0.8071074  0.         0.         0.        ]
 [0.42149907 0.86134565 0.         0.         0.        ]
 [0.24827704 0.6849937  0.         0.         0.        ]
 [0.33513385 1.1609813  0.         0.         0.        ]
 [0.54836035 1.18244    0.         0.         0.        ]]

[0.98516953 0.6033745  0.         0.         0.        ]


In [32]:
print(ode_data_list)

[[[ 1.0091585   0.6240594   0.11126312 -0.36589593 -0.7523089
   -1.2567445  -1.483704   -2.138241   -2.489428   -2.8319573 ]
  [ 1.1297613   1.128918    1.2535115   1.5820136   1.6348484
    1.6701912   1.6949056   1.7714494   1.9393784   1.9313073 ]
  [ 0.          0.22263974  0.41280398  0.6925523   0.9023686
    1.0331072   1.2210448   1.5210226   1.7961234   2.0147576 ]
  [ 0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]]

 [[ 0.9643247   0.59033537  0.11804659 -0.27548742 -0.75462174
   -1.171533   -1.4770889  -2.1577663  -2.6474888  -2.9693894 ]
  [ 0.5252658   0.7118832   0.85274625  0.9429426   1.0948491
    1.0404936   1.3151113   1.2455858   1.4411094   1.4325341 ]
  [ 0.          0.20649207  0.41733244  0.66269934  0.83007294
    1.1461535   1.2664899   1.5725093   1.7740191   1.9276471 ]
  

###  Нужно еще более глубоко проверить, и сравнить поэтапно, то ли выполняется! 
### На данный момент получилось запустить, вроде бы работает как надо!